# Control 3

Para este control trabajaremos sobre un dataset de best sellers del New York Times, que tiene información sobre libros que han estado en la lista.

Pon tu nombre acá: ____Jorge De Goyeneche____

In [ ]:
!wget https://github.com/IIC2413/Syllabus-2022-2/raw/main/Controles/Control%203/nyt2.json

!apt install mongodb
!service mongodb start
!mongoimport --db local --collection nyt --jsonArray < nyt2.json

In [ ]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names()
db = client['local']
collection = db['nyt']

In [ ]:
import pprint
for book in collection.find({}).limit(1):
    pprint.pprint(book)

{'_id': ObjectId('5b4aa4ead3089013507db18b'),
 'amazon_product_url': 'http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20',
 'author': 'Dean R Koontz',
 'bestsellers_date': datetime.datetime(2008, 5, 24, 0, 0),
 'description': 'Odd Thomas, who can communicate with the dead, confronts evil '
                'forces in a California coastal town.',
 'price': 27,
 'published_date': datetime.datetime(2008, 6, 8, 0, 0),
 'publisher': 'Bantam',
 'rank': 1,
 'rank_last_week': 0,
 'title': 'ODD HOURS',
 'weeks_on_list': 1}


# NoSQL

## Pregunta 1

Encuentre los libros que han estado más de 10 semanas en la lista.

**Codigo comentado lo utilicé para filtrar duplicados**

In [ ]:
#libros = []
for book in collection.find({'weeks_on_list': { '$gt': 10 }}):
  #if (book['publisher'], book['author'], book['title']) not in libros:
  print(f"EDITORIAL: {book['publisher']}, AUTOR: {book['author']}, TITULO: {book['title']}, SEMANAS: {book['weeks_on_list']}")
    #libros.append((book['publisher'], book['author'], book['title']))


## Pregunta 2

Encuentre los libros anteriores al 2015 que estuvieron más de 5 semanas en la lista de best sellers, indique cuantas semanas estuvo cada libro en la lista.

In [ ]:
import datetime
#libros = []
for book in collection.find({'weeks_on_list': { '$gt': 5 }, 'published_date': {'$lt': datetime.datetime(2015,1,1,0,0)}}).sort('weeks_on_list', -1):
  #if (book["title"], book['author'], book['publisher']) not in libros:
  print(f'SEMANAS: {book["weeks_on_list"]}, TITULO: {book["title"]}, AÑO: {book["published_date"].year}')
    #libros.append((book["title"], book['author'], book['publisher']))

SEMANAS: 108, TITULO: THE HELP, AÑO: 2011
SEMANAS: 86, TITULO: A DANCE WITH DRAGONS, AÑO: 2013
SEMANAS: 80, TITULO: GONE GIRL, AÑO: 2014
SEMANAS: 79, TITULO: THE GIRL WHO KICKED THE HORNET’S NEST, AÑO: 2012
SEMANAS: 59, TITULO: THE HOST, AÑO: 2009
SEMANAS: 55, TITULO: THE GOLDFINCH, AÑO: 2014
SEMANAS: 39, TITULO: THE STORY OF EDGAR SAWTELLE, AÑO: 2009
SEMANAS: 33, TITULO: THE GUERNSEY LITERARY AND POTATO PEEL PIE SOCIETY, AÑO: 2009
SEMANAS: 32, TITULO: ALL THE LIGHT WE CANNOT SEE, AÑO: 2014
SEMANAS: 31, TITULO: THE PARIS WIFE, AÑO: 2012
SEMANAS: 29, TITULO: THE LOST SYMBOL, AÑO: 2010
SEMANAS: 29, TITULO: INFERNO, AÑO: 2014
SEMANAS: 26, TITULO: THE HUSBAND'S SECRET, AÑO: 2014
SEMANAS: 25, TITULO: THE INVENTION OF WINGS, AÑO: 2014
SEMANAS: 24, TITULO: AND THE MOUNTAINS ECHOED, AÑO: 2014
SEMANAS: 23, TITULO: DEAD AND GONE, AÑO: 2009
SEMANAS: 22, TITULO: THE LAST SONG, AÑO: 2010
SEMANAS: 22, TITULO: SYCAMORE ROW, AÑO: 2014
SEMANAS: 21, TITULO: THE GIRL WHO PLAYED WITH FIRE, AÑO: 2010
SEMAN

## Pregunta 3

Muestre el raking de los editores (publishers) que más semanas han estado en la lista de los best sellers.

In [174]:
import pymongo
group = {
   "$group": {
       "_id": "$publisher",
       "weeks_on_list": {"$sum": 1}
       }
}
sort = {
  "$sort": {"weeks_on_list": pymongo.DESCENDING}
}

pipeline = [
   group,
   sort
]
resultados = collection.aggregate(pipeline)

i = 1
for resultado in resultados:
   print(f"{i} SEMANAS: {resultado['weeks_on_list']} EDITORIAL: {resultado['_id']}")
   i += 1

1 SEMANAS: 1061 EDITORIAL: Putnam
2 SEMANAS: 943 EDITORIAL: Little, Brown
3 SEMANAS: 619 EDITORIAL: Grand Central
4 SEMANAS: 559 EDITORIAL: Knopf
5 SEMANAS: 475 EDITORIAL: Doubleday
6 SEMANAS: 456 EDITORIAL: Scribner
7 SEMANAS: 421 EDITORIAL: Ballantine
8 SEMANAS: 347 EDITORIAL: Delacorte
9 SEMANAS: 319 EDITORIAL: Bantam
10 SEMANAS: 308 EDITORIAL: Simon & Schuster
11 SEMANAS: 301 EDITORIAL: St. Martin's
12 SEMANAS: 269 EDITORIAL: Viking
13 SEMANAS: 252 EDITORIAL: Random House
14 SEMANAS: 240 EDITORIAL: Morrow/HarperCollins
15 SEMANAS: 231 EDITORIAL: Dutton
16 SEMANAS: 211 EDITORIAL: Harper
17 SEMANAS: 199 EDITORIAL: Riverhead
18 SEMANAS: 185 EDITORIAL: Amy Einhorn/Putnam
19 SEMANAS: 183 EDITORIAL: Atria
20 SEMANAS: 178 EDITORIAL: St. Martin’s
21 SEMANAS: 165 EDITORIAL: Harper/HarperCollins
22 SEMANAS: 149 EDITORIAL: Crown
23 SEMANAS: 125 EDITORIAL: Ace
24 SEMANAS: 100 EDITORIAL: Morrow
25 SEMANAS: 82 EDITORIAL: Emily Bestler/Atria
26 SEMANAS: 79 EDITORIAL: Penguin Press
27 SEMANAS: 74 

# Pandas

Resuelva las preguntas anteriores usando Pandas, en vez de consultar directamente en MongoDb

In [175]:
import pandas as pd
df = pd.read_json('nyt2.json')
df.head()

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list
0,{'$oid': '5b4aa4ead3089013507db18b'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",{'$numberInt': '27'},Bantam,ODD HOURS,{'$numberInt': '1'},{'$numberInt': '0'},{'$numberInt': '1'}
1,{'$oid': '5b4aa4ead3089013507db18c'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,{'$numberDouble': '25.99'},"Little, Brown",THE HOST,{'$numberInt': '2'},{'$numberInt': '1'},{'$numberInt': '3'}
2,{'$oid': '5b4aa4ead3089013507db18d'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,{'$numberDouble': '24.95'},St. Martin's,LOVE THE ONE YOU'RE WITH,{'$numberInt': '3'},{'$numberInt': '2'},{'$numberInt': '2'}
3,{'$oid': '5b4aa4ead3089013507db18e'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,{'$numberDouble': '22.95'},Putnam,THE FRONT,{'$numberInt': '4'},{'$numberInt': '0'},{'$numberInt': '1'}
4,{'$oid': '5b4aa4ead3089013507db18f'},{'$date': {'$numberLong': '1211587200000'}},{'$date': {'$numberLong': '1212883200000'}},http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,{'$numberDouble': '24.95'},Doubleday,SNUFF,{'$numberInt': '5'},{'$numberInt': '0'},{'$numberInt': '1'}


**Limpieza De Datos**

In [176]:
df['_id'] = pd.DataFrame.from_records(df['_id'])
df['rank'] = pd.DataFrame.from_records(df['rank'])
df['rank'] = df['rank'].astype(int)

df['rank_last_week'] = pd.DataFrame.from_records(df['rank_last_week'])
df['rank_last_week'] = df['rank_last_week'].astype(int)

df['published_date'] = pd.DataFrame.from_records(df['published_date'])
df['published_date'] = pd.DataFrame.from_records(df['published_date'])
df['published_date'] = df['published_date'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1e3))

df['bestsellers_date'] = pd.DataFrame.from_records(df['bestsellers_date'])
df['bestsellers_date'] = pd.DataFrame.from_records(df['bestsellers_date'])
df['bestsellers_date'] = df['bestsellers_date'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1e3))

df['weeks_on_list'] = pd.DataFrame.from_records(df['weeks_on_list'])
df['weeks_on_list'] = df['weeks_on_list'].astype(int)

df['published_year'] = df['published_date'].apply(lambda x: x.year)

df

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list,published_year
0,5b4aa4ead3089013507db18b,2008-05-24,2008-06-08,http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"Odd Thomas, who can communicate with the dead,...",{'$numberInt': '27'},Bantam,ODD HOURS,1,0,1,2008
1,5b4aa4ead3089013507db18c,2008-05-24,2008-06-08,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,{'$numberDouble': '25.99'},"Little, Brown",THE HOST,2,1,3,2008
2,5b4aa4ead3089013507db18d,2008-05-24,2008-06-08,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman's happy marriage is shaken when she en...,{'$numberDouble': '24.95'},St. Martin's,LOVE THE ONE YOU'RE WITH,3,2,2,2008
3,5b4aa4ead3089013507db18e,2008-05-24,2008-06-08,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,A Massachusetts state investigator and his tea...,{'$numberDouble': '22.95'},Putnam,THE FRONT,4,0,1,2008
4,5b4aa4ead3089013507db18f,2008-05-24,2008-06-08,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,An aging porn queens aims to cap her career by...,{'$numberDouble': '24.95'},Doubleday,SNUFF,5,0,1,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10190,5b4aa4ead3089013507dd959,2018-07-07,2018-07-22,https://www.amazon.com/Clancy-Line-Sight-Jack-...,Mike Maden,Jack Ryan Jr. risks his life to protect a woma...,{'$numberInt': '0'},Putnam,TOM CLANCY LINE OF SIGHT,11,6,4,2018
10191,5b4aa4ead3089013507dd95a,2018-07-07,2018-07-22,https://www.amazon.com/Something-Water-Novel-C...,Catherine Steadman,A documentary filmmaker and an investment bank...,{'$numberInt': '0'},Ballantine,SOMETHING IN THE WATER,12,11,5,2018
10192,5b4aa4ead3089013507dd95b,2018-07-07,2018-07-22,https://www.amazon.com/Little-Fires-Everywhere...,Celeste Ng,An artist upends a quiet town outside Cleveland.,{'$numberInt': '0'},Penguin Press,LITTLE FIRES EVERYWHERE,13,12,41,2018
10193,5b4aa4ead3089013507dd95c,2018-07-07,2018-07-22,https://www.amazon.com/Shelter-Place-Nora-Robe...,Nora Roberts,Survivors of a mass shooting outside a mall in...,{'$numberInt': '0'},St. Martin's,SHELTER IN PLACE,14,5,6,2018


Para limpiar el precio:


In [164]:
for i in range(len(df)):
  if 'Int' in str(df.loc[i,'price']):
    df.loc[i,'price'] = int(df.loc[i, 'price'].get('$numberInt'))
  else:
    df.loc[i,'price'] = float(df.loc[i, 'price'].get('$numberDouble'))

## Pregunta 4

Encuentre los libros que han estado más de 10 semanas en la lista.

In [177]:
df.copy()[df['weeks_on_list'] > 10]

,_id,bestsellers_date,published_date,amazon_product_url,author,description,price,publisher,title,rank,rank_last_week,weeks_on_list,published_year
163,5b4aa4ead3089013507db22e,2008-07-19,2008-08-03,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,{'$numberDouble': '25.99'},"Little, Brown",THE HOST,4,7,11,2008
182,5b4aa4ead3089013507db241,2008-07-26,2008-08-10,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,{'$numberDouble': '25.99'},"Little, Brown",THE HOST,3,4,12,2008
194,5b4aa4ead3089013507db24d,2008-07-26,2008-08-10,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman’s happy marriage is shaken when she en...,{'$numberDouble': '24.95'},St. Martin’s,LOVE THE ONE YOU'RE WITH,15,13,11,2008
202,5b4aa4ead3089013507db255,2008-08-02,2008-08-17,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Aliens have taken control of the minds and bod...,{'$numberDouble': '25.99'},"Little, Brown",THE HOST,3,3,13,2008
213,5b4aa4ead3089013507db260,2008-08-02,2008-08-17,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,A woman encounters an old boyfriend.,{'$numberDouble': '24.95'},St. Martin’s,LOVE THE ONE YOU'RE WITH,14,15,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10178,5b4aa4ead3089013507dd94d,2018-06-30,2018-07-15,https://www.amazon.com/Woman-Window-Novel-J-Fi...,AJ Finn,A recluse who drinks heavily and takes prescri...,{'$numberInt': '0'},Morrow,THE WOMAN IN THE WINDOW,14,0,23,2018
10179,5b4aa4ead3089013507dd94e,2018-06-30,2018-07-15,https://www.amazon.com/Great-Alone-Novel-Krist...,Kristin Hannah,A former prisoner of war moves his family to A...,{'$numberInt': '0'},St. Martin's,THE GREAT ALONE,15,0,18,2018
10188,5b4aa4ead3089013507dd957,2018-07-07,2018-07-22,https://www.amazon.com/Fallen-Memory-Man-David...,David Baldacci,Amos Decker puts his talents toward solving a ...,{'$numberInt': '0'},Grand Central,THE FALLEN,9,13,12,2018
10189,5b4aa4ead3089013507dd958,2018-07-07,2018-07-22,https://www.amazon.com/Before-We-Were-Yours-No...,Lisa Wingate,A South Carolina lawyer learns about the quest...,{'$numberInt': '0'},Ballantine,BEFORE WE WERE YOURS,10,10,42,2018


## Pregunta 5

Encuentre los libros anteriores al 2015 que estuvieron más de 5 semanas en la lista de best sellers, indique cuantas semanas estuvo cada libro en la lista. (Los campos de fechas vienen como timestamps de UNIX)

In [178]:
df.copy().loc[df['weeks_on_list'] > 5].loc[df['published_year'] < 2015][['title', 'weeks_on_list', 'published_year']]

,title,weeks_on_list,published_year
32,THE WHOLE TRUTH,6,2008
47,SUNDAYS AT TIFFANY’S,6,2008
62,THE HOST,6,2008
69,SUNDAYS AT TIFFANY’S,7,2008
82,THE HOST,7,2008
...,...,...,...
6847,THE BURNING ROOM,6,2014
6848,EDGE OF ETERNITY,13,2014
6850,PRINCE LESTAT,7,2014
6852,THE GOLDFINCH,55,2014


# Pregunta 6

Muestre el raking de los editores (publishers) que más semanas han estado en la lista de los best sellers.

In [179]:
df['publisher'].value_counts().reset_index()

,index,publisher
0,Putnam,1061
1,"Little, Brown",943
2,Grand Central,619
3,Knopf,559
4,Doubleday,475
...,...,...
171,Morrow/HarperCollins Publishers,1
172,ECW Press,1
173,Harlequin HQN,1
174,Harper Voyager/HarperCollins Publishers,1
